In [1]:
### fix the links for prev and next pages

import os
import numpy as np
import pandas as pd

path = './'

ALL_PAGE_FILE_PREFIX = 'page'

def extract_page_number (filename):
    return int(filename.replace(ALL_PAGE_FILE_PREFIX, '')[0:3])


### only include pages in these
ls_filename = []
ls_filenum  = []

### find all html page files
for root, dirs, files in os.walk(path):
    if root == path:
        ### get a list of page numbers of all presentation pages,
        ### not including plot files etc.
        for name in files:
            if ALL_PAGE_FILE_PREFIX in name[0:len(ALL_PAGE_FILE_PREFIX)]:
                ls_filename.append(name)
                ls_filenum.append(extract_page_number(name))

### add "the_end.html" at the end
ls_filename.append('the_end.html')
ls_filenum.append(99999)

sr_filename = pd.Series(ls_filename, name='filename')
sr_filenum  = pd.Series(ls_filenum,  name='filenum')

df_all_page_files = pd.DataFrame(sr_filename)
df_all_page_files = df_all_page_files.merge(sr_filenum, how='outer', left_index=True, right_index=True)

df_all_page_files = df_all_page_files.sort_values('filenum', ascending=True).reset_index(drop=True)

def fix_one_html_file (row_idx : int):
    global df_all_page_files

    filename = df_all_page_files.iloc[row_idx].loc['filename']

    filename_prev = '#'
    filename_next = 'the_end.html'

    ### except first file
    if row_idx != 0:
        filename_prev = df_all_page_files.iloc[row_idx - 1].loc['filename']

    ### except last file
    if row_idx != df_all_page_files.index.max():
        filename_next = df_all_page_files.iloc[row_idx + 1].loc['filename']

    s = open(filename, 'rt', encoding='utf-8').read()

    s1 = s

    n_idx_id_prev_page = s1.find('id="prev_page">')
    n_idx_href_prev_page = s1.rfind('href=', 0, n_idx_id_prev_page)
    s1 = s1[0:n_idx_href_prev_page] + f'href="{filename_prev}" ' + s1[n_idx_id_prev_page:]

    n_idx_id_next_page = s1.find('id="next_page">')
    n_idx_href_next_page = s1.rfind('href=', 0, n_idx_id_next_page)
    s1 = s1[0:n_idx_href_next_page] + f'href="{filename_next}" ' + s1[n_idx_id_next_page:]

    if s1 != s:
        with open(filename, 'wt', encoding='utf-8') as f:
            f.write(s1)


f1 = np.vectorize(fix_one_html_file, cache=True)(df_all_page_files.index)

#fix_one_html_file(0)


./
./img
